### 1. Import Libraries & Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df0 = pd.read_csv("../../C_Datasets/titanic_data_train.csv")
df = df0.copy()
df.head()

,Sex,Pclass,SibSp,Parch,Alone,Embarked,Age,Ticket,Fare,Survived
0,1,3,1,0,1,2,22.0,21171,7.2500,0
1,0,1,1,0,1,0,38.0,17599,71.2833,1
2,0,3,0,0,0,2,26.0,3101282,7.9250,1
3,0,1,1,0,1,2,35.0,113803,53.1000,1
4,1,3,0,0,0,2,35.0,373450,8.0500,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sex       891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Alone     891 non-null    int64  
 5   Embarked  891 non-null    int64  
 6   Age       891 non-null    float64
 7   Ticket    891 non-null    int64  
 8   Fare      891 non-null    float64
 9   Survived  891 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 69.7 KB


In [4]:
df_clf = df.drop(columns=["SibSp", "Parch", "Embarked"])
df_clf.head()

,Sex,Pclass,Alone,Age,Ticket,Fare,Survived
0,1,3,1,22.0,21171,7.2500,0
1,0,1,1,38.0,17599,71.2833,1
2,0,3,0,26.0,3101282,7.9250,1
3,0,1,1,35.0,113803,53.1000,1
4,1,3,0,35.0,373450,8.0500,0


### 2. Standardization

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
df_clf.loc[:, "Age"] = scaler.fit_transform(df_clf.iloc[:, 3:-1])[:, 0]
df_clf.loc[:, "Ticket"] = scaler.fit_transform(df_clf.iloc[:, 3:-1])[:, 1]
df_clf.loc[:, "Fare"] = scaler.fit_transform(df_clf.iloc[:, 3:-1])[:, 2]

In [7]:
df_clf.head()

,Sex,Pclass,Alone,Age,Ticket,Fare,Survived
0,1,3,1,-0.551366,-0.423103,-0.502445,0
1,0,1,1,0.654030,-0.428495,0.786845,1
2,0,3,0,-0.250017,4.226398,-0.488854,1
3,0,1,1,0.428018,-0.283273,0.420730,1
4,1,3,0,0.428018,0.108670,-0.486337,0


### 3. Split Data Frame to Train & Test Set

In [8]:
from sklearn.model_selection import train_test_split

X = df_clf.drop(columns=["Survived"]).values
y = df_clf.loc[:, "Survived"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### 4. Create Model

In [9]:
from sklearn.linear_model import LogisticRegression

log_reg_model = LogisticRegression().fit(X_train, y_train)

In [10]:
y_pred = log_reg_model.predict(X_test)

### 5. Evaluation

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [13]:
pd.DataFrame({"Model": "Logistic Regression Classifier",
              "Accuracy": accuracy,
              "Precision": precision,
              "Recall": recall,
              "F1-Score": f1},
            index = [0])

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression Classifier,0.782123,0.727273,0.695652,0.711111


### 6. Optimization

In [41]:
from sklearn.model_selection import GridSearchCV

parameters = {'solver': ['saga', 'liblinear'],
              'penalty':['l1'],
              'C':[0.001, 0.01, 0.1, 1, 10, 100]}

log_reg_model_grid_1 = GridSearchCV(LogisticRegression(), parameters)

parameters = {'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
              'penalty':['l2'],
              'C':[0.001, 0.01, 0.1, 1, 10, 100]}

log_reg_model_grid_2 = GridSearchCV(LogisticRegression(), parameters)

In [42]:
log_reg_model_grid_1.fit(X_train, y_train)
log_reg_model_grid_2.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']})

In [43]:
log_reg_model_grid_1.best_params_

{'C': 100, 'penalty': 'l1', 'solver': 'saga'}

In [45]:
log_reg_model = LogisticRegression(C=100, penalty='l1', solver="saga").fit(X_train, y_train)
y_pred = log_reg_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

pd.DataFrame({"Model": "Logistic Regression Classifier",
              "Accuracy": accuracy,
              "Precision": precision,
              "Recall": recall,
              "F1-Score": f1},
            index = [0])

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression Classifier,0.782123,0.727273,0.695652,0.711111


In [44]:
log_reg_model_grid_2.best_params_

{'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}

In [46]:
log_reg_model = LogisticRegression(C=1, penalty='l2', solver="lbfgs").fit(X_train, y_train)
y_pred = log_reg_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

pd.DataFrame({"Model": "Logistic Regression Classifier",
              "Accuracy": accuracy,
              "Precision": precision,
              "Recall": recall,
              "F1-Score": f1},
            index = [0])

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression Classifier,0.782123,0.727273,0.695652,0.711111
